In [1]:
%reload_ext autoreload
%autoreload all

%aimport -torch
%aimport -matplotlib
%aimport -seaborn
%aimport -numpy
%aimport -pandas
%aimport -scipy
%aimport -lightning 

In [2]:
from eeg_snn_encoder.config import PROCESSED_DATA_DIR
from eeg_snn_encoder.dataset import CHBMITDataset, CHBMITDataModule

# Load the dataset
dataset = CHBMITDataset(PROCESSED_DATA_DIR / "stft_normalized.h5")

2025-05-07 12:19:36.048 | INFO     | eeg_snn_encoder.config:<module>:11 - PROJ_ROOT path is: /root/snn-encoder-test


In [3]:
datamodule = CHBMITDataModule(dataset, batch_size=512, worker=0)

In [4]:
import os

import optuna

# Initialize the Optuna study
sampler = optuna.samplers.TPESampler(multivariate=True)
study = optuna.load_study(
    study_name="Classifier_PE_Tuning",
    storage=os.environ["OPTUNA_CONN_STRING"],
    sampler=sampler,
)

/root/snn-encoder-test/.venv/lib/python3.12/site-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [5]:
complete_trial = study.get_trials(False, states=(optuna.trial.TrialState.COMPLETE,))
sorted_trials = sorted(complete_trial, key=lambda t: t.value)
filtered_trials = list(filter(lambda t: t.value < 16, sorted_trials))
filtered_trials

[FrozenTrial(number=92, state=1, values=[8.969496726989746], datetime_start=datetime.datetime(2025, 5, 1, 23, 19, 21, 582101), datetime_complete=datetime.datetime(2025, 5, 1, 23, 23, 31, 34055), params={'threshold': 0.08749666342500385, 'slope': 10.517967728531268, 'beta': 0.7429051971933321, 'dropout_rate1': 0.739827893137529, 'dropout_rate2': 0.38463703727491416, 'lr': 3.2548315250403596e-05, 'weight_decay': 1.449218153299157e-05, 'scheduler_factor': 0.22126715557391818, 'scheduler_patience': 6, 'phase_window': 1}, user_attrs={}, system_attrs={'cma:generation': 3, 'cma:n_restarts': 0, 'cma:poptype': 'small', 'cma:large_n_eval': 0, 'completed_rung_0': 9.995233535766602, 'completed_rung_1': 14.477381706237793, 'cma:popsize': 10, 'cma:n_restarts_with_large': 0, 'cma:small_n_eval': 0}, intermediate_values={1: 10.951603889465332, 3: 9.995233535766602, 5: 9.390844345092773, 7: 9.013471603393555, 9: 14.477381706237793, 11: 9.05643367767334, 13: 9.099499702453613, 0: 13.774213790893555, 2: 1

In [6]:
from loguru import logger

import lightning.pytorch as pl
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from optuna.integration import PyTorchLightningPruningCallback

from eeg_snn_encoder.encoders import PhaseEncoderExpand
from eeg_snn_encoder.models.classifier import ModelConfig
from eeg_snn_encoder.models.lightning import LitSeizureClassifier, OptimizerConfig
import torch

torch.set_float32_matmul_precision('medium')


def objective(trial: optuna.Trial) -> float:
    model_params: ModelConfig = {
        "threshold": trial.suggest_float("threshold", 0.01, 0.5),
        "slope": trial.suggest_float("slope", 1.0, 20.0),
        "beta": trial.suggest_float("beta", 0.1, 0.99),
        "dropout_rate1": trial.suggest_float("dropout_rate1", 0.1, 0.99),
        "dropout_rate2": trial.suggest_float("dropout_rate2", 0.1, 0.99),
    }

    optimizer_params: OptimizerConfig = {
        "lr": trial.suggest_float("lr", 1e-6, 1e-4, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 1e-6, 1e-4, log=True),
        "scheduler_factor": trial.suggest_float("scheduler_factor", 0.1, 0.99),
        "scheduler_patience": trial.suggest_int("scheduler_patience", 1, 10),
    }

    encoder_params = {
        "phase_window": trial.suggest_int("phase_window", 1, 8),
    }

    spike_encoder = PhaseEncoderExpand(**encoder_params)
    
    lit_model = LitSeizureClassifier(
        model_config=model_params,
        optimizer_config=optimizer_params,
        spike_encoder=spike_encoder,
    )

    trainer = pl.Trainer(
        max_epochs=50,
        accelerator="auto",
        devices="auto",
        strategy="auto",
        enable_model_summary=False,
        enable_checkpointing=False,
        callbacks=[PyTorchLightningPruningCallback(trial, monitor="val_loss"), EarlyStopping(monitor="val_loss", mode="min", patience=5)],
        logger=False,
    )

    trainer.fit(lit_model, datamodule=datamodule)

    val_loss = trainer.callback_metrics["val_loss"].item()
    
    trainer.test(lit_model, datamodule=datamodule)

    test_loss = trainer.callback_metrics["test_loss"].item()
    test_acc = trainer.callback_metrics["test_acc"]
    test_f1 = trainer.callback_metrics["test_f1"]
    test_mse = trainer.callback_metrics["test_mse"]
    test_total_spikes = trainer.callback_metrics["test_total_spikes"]

    logger.info(f"Encoder: Burst Encoding,trial: {trial.number}, test_loss:{test_loss}, test_mse:{test_mse}, test_acc:{test_acc}, test_f1:{test_f1}, test_total_spikes:{test_total_spikes}")

    return val_loss

In [7]:
for trial in filtered_trials:
    study.enqueue_trial(params=trial.params)

study.optimize(objective, n_trials=len(filtered_trials))

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/root/snn-encoder-test/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=47` in the `DataLoader` to improve performance.
/root/snn-encoder-test/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=47` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.8231382966041565     │
│          test_f1          │    0.8373470902442932     │
│         test_loss         │     9.445549964904785     │
│         test_mse          │    0.1768617033958435     │
│      test_precision       │    0.8032165765762329     │
│        test_recall        │    0.8746413588523865     │
│     test_total_spikes     │      29924.39453125       │
└───────────────────────────┴───────────────────────────┘

2025-05-07 12:35:45.204 | INFO     | __main__:objective:66 - Encoder: Burst Encoding,trial: 113, test_loss:9.445549964904785, test_mse:0.1768617033958435, test_acc:0.8231382966041565, test_f1:0.8373470902442932, test_total_spikes:29924.39453125


[I 2025-05-07 12:35:46,385] Trial 113 finished with value: 9.472602844238281 and parameters: {'threshold': 0.08749666342500385, 'slope': 10.517967728531268, 'beta': 0.7429051971933321, 'dropout_rate1': 0.739827893137529, 'dropout_rate2': 0.38463703727491416, 'lr': 3.2548315250403596e-05, 'weight_decay': 1.449218153299157e-05, 'scheduler_factor': 0.22126715557391818, 'scheduler_patience': 6, 'phase_window': 1}. Best is trial 92 with value: 8.969496726989746.


In [9]:
logger.info(f"Encoder: Burst Encoding,trial, best_param: {study.best_params}")
logger.info(f"Encoder: Burst Encoding,trial, best_score: {study.best_value}")

2025-05-07 13:04:54.205 | INFO     | __main__:<module>:1 - Encoder: Burst Encoding,trial, best_param: {'threshold': 0.08749666342500385, 'slope': 10.517967728531268, 'beta': 0.7429051971933321, 'dropout_rate1': 0.739827893137529, 'dropout_rate2': 0.38463703727491416, 'lr': 3.2548315250403596e-05, 'weight_decay': 1.449218153299157e-05, 'scheduler_factor': 0.22126715557391818, 'scheduler_patience': 6, 'phase_window': 1}
2025-05-07 13:04:54.626 | INFO     | __main__:<module>:2 - Encoder: Burst Encoding,trial, best_score: 8.969496726989746
